In [2]:
import gradio as gr

# A global variable to store food entries for the current session.
food_log = []

def calculate_bmi(unit_system, weight, height):
    """Calculates BMI and returns the result and category."""
    if not weight or not height or weight <= 0 or height <= 0:
        return "Please enter valid weight and height.", ""

    if unit_system == "Imperial (lb, in)":
        weight = weight * 0.453592
        height = height * 2.54

    height_m = height / 100
    bmi = weight / (height_m ** 2)

    category = ""
    if bmi < 18.5:
        category = "Underweight"
    elif 18.5 <= bmi < 25:
        category = "Normal weight"
    elif 25 <= bmi < 30:
        category = "Overweight"
    else:
        category = "Obesity"

    return f"Your BMI is: {bmi:.2f}", f"Category: {category}"

def calculate_metabolic_rate(age, gender, weight, height, activity_level):
    """Calculates BMR and estimated daily caloric needs using the Mifflin-St Jeor formula."""
    if not age or not weight or not height or weight <= 0 or height <= 0 or age <= 0:
        return "Please enter valid age, weight, and height."

    if gender == "male":
        bmr = (10 * weight) + (6.25 * height) - (5 * age) + 5
    else:
        bmr = (10 * weight) + (6.25 * height) - (5 * age) - 161

    # The activity_level variable is now the numerical value directly
    daily_calories = bmr * activity_level

    return (f"Your Basal Metabolic Rate (BMR) is: {bmr:.2f} kcal/day.\n"
            f"Your estimated daily calorie needs are: {daily_calories:.2f} kcal/day.")

def add_food_entry(food_item, calories):
    """Adds a food entry and returns the updated log and total calories."""
    if not food_item or not calories or calories < 0:
        return "Please enter a valid food item and calorie count.", [], 0

    food_log.append({"item": food_item, "calories": calories})

    log_output = [[entry['item'], entry['calories']] for entry in food_log]
    total_calories = sum(entry['calories'] for entry in food_log)

    return "Food added!", log_output, total_calories

def clear_food_log():
    """Clears the food log."""
    global food_log
    food_log = []
    return "Log cleared!", [], 0

# Define a custom theme to make colors similar to the image
custom_theme = gr.themes.Soft(
    primary_hue="cyan",
).set(
    button_primary_background_fill="#77c2c7",
    button_primary_background_fill_hover="#18acba",
    button_secondary_background_fill="white",
    button_secondary_background_fill_hover="#e8f9fa",
    block_background_fill="white",
    block_border_width="1px",
    block_border_color="#d5eced",
    block_shadow="0 1px 3px 0 rgb(0 0 0 / 0.1), 0 1px 2px -1px rgb(0 0 0 / 0.1)",
    body_background_fill="#f0f2f5",
)

# Create the Gradio interface using Blocks for tabs
with gr.Blocks(title="BME Health Calculator Suite", theme=custom_theme, css="""
    .main-container {
        max-width: 800px;
        margin: 0 auto;
        padding: 0;
    }
    .header-section {
        background-color: #a8e1e6;
        padding: 20px;
        text-align: center;
        color: white;
        border-radius: 8px 8px 0 0;
    }
    .header-section h1 {
        font-size: 2em;
        margin-bottom: 5px;
        display: flex;
        align-items: center;
        justify-content: center;
        gap: 10px;
    }
    .header-section h1 img {
        height: 40px;
        vertical-align: middle;
    }
    .header-section p {
        font-size: 1.1em;
        margin: 0;
    }
    .gradio-tabs {
        border-radius: 8px;
        background-color: transparent;
        border: none;
        box-shadow: none;
        margin-top: 20px;
    }
    .gradio-tabs > div[role="tablist"] {
        justify-content: center;
    }
    .gradio-tabs > div[role="tablist"] > button {
        background-color: transparent !important;
        color: #18afba !important;
        font-size: 1.1em;
        font-weight: bold;
        border: none !important;
        border-bottom: 2px solid transparent !important;
        padding: 15px 30px;
        position: relative;
    }
    .gradio-tabs > div[role="tablist"] > button.selected {
        background-color: transparent !important;
        color: #18afba !important;
        border-bottom-color: #18afba !important;
    }
    .gradio-tabs .gradio-tab-content {
        padding: 20px;
        background-color: white;
        border-radius: 8px;
        box-shadow: 0 1px 3px 0 rgb(0 0 0 / 0.1), 0 1px 2px -1px rgb(0 0 0 / 0.1);
        margin-top: 20px;
    }
""") as demo:
    gr.Markdown(f"""
    <div class="header-section">
        <h1>
            BME Health Calculator Suite
        </h1>
        <p>A comprehensive health tracking application for Biomedical Engineering students</p>
    </div>
    """)

    with gr.Tabs(elem_classes=["gradio-tabs"]):
        # BMI Calculator Tab
        with gr.Tab("📊 BMI Calculator"):
            with gr.Group(elem_classes=["gradio-tab-content"]):
                gr.Markdown("### <font color='#34A853'>**Body Mass Index Calculator**</font>")
                gr.Markdown("Calculate your BMI and see where you fall on the health categories.")

                bmi_unit_system = gr.Dropdown(
                    label="Unit System:",
                    choices=["Metric (cm, kg)", "Imperial (lb, in)"],
                    value="Metric (cm, kg)"
                )

                bmi_height_input = gr.Number(label="Height (cm):", placeholder="e.g., 170")
                bmi_weight_input = gr.Number(label="Weight (kg):", placeholder="e.g., 70")

                bmi_calculate_btn = gr.Button("Calculate BMI", variant="primary")

                with gr.Column():
                    bmi_result_output = gr.Label(label="Your BMI")
                    bmi_category_output = gr.Label(label="Category")

                bmi_calculate_btn.click(
                    fn=calculate_bmi,
                    inputs=[bmi_unit_system, bmi_weight_input, bmi_height_input],
                    outputs=[bmi_result_output, bmi_category_output]
                )

        # Metabolic Rate Calculator Tab
        with gr.Tab("🔥 Metabolic Rate"):
            with gr.Group(elem_classes=["gradio-tab-content"]):
                gr.Markdown("### <font color='#2c888f'>**Metabolic Rate Calculator**</font>")
                gr.Markdown("Calculate your estimated daily calorie needs.")
                with gr.Row():
                    mr_age_input = gr.Number(label="Age")
                    mr_gender_input = gr.Radio(["male", "female"], label="Gender")
                with gr.Row():
                    mr_weight_input = gr.Number(label="Weight (kg)")
                    mr_height_input = gr.Number(label="Height (cm)")

                # FIX: Separate choices from values to pass numerical data directly
                activity_levels = [1.2, 1.375, 1.55, 1.725, 1.9]
                activity_labels = ["Sedentary", "Lightly Active", "Moderately Active", "Very Active", "Extra Active"]
                mr_activity_input = gr.Radio(
                    label="Activity Level",
                    choices=activity_labels,
                    value=activity_labels[0]
                )

                mr_calculate_btn = gr.Button("Calculate Calories", variant="primary")
                mr_result_output = gr.Label(label="Metabolic Rate and Daily Calories")

                def get_calories(age, gender, weight, height, activity_label):
                    # Map the label back to the numerical value
                    activity_multiplier = activity_levels[activity_labels.index(activity_label)]
                    return calculate_metabolic_rate(age, gender, weight, height, activity_multiplier)

                mr_calculate_btn.click(
                    fn=get_calories,
                    inputs=[mr_age_input, mr_gender_input, mr_weight_input, mr_height_input, mr_activity_input],
                    outputs=mr_result_output
                )

        # Food Tracker Tab
        with gr.Tab("🍎 Food Tracker"):
            with gr.Group(elem_classes=["gradio-tab-content"]):
                gr.Markdown("### <font color='#34A853'>**Food Tracker**</font>")
                gr.Markdown("Track your daily calorie intake.")
                with gr.Row():
                    food_item_input = gr.Textbox(label="Food Item")
                    food_calories_input = gr.Number(label="Calories (kcal)")

                add_food_btn = gr.Button("Add Food", variant="primary")

                with gr.Column():
                    food_log_list = gr.DataFrame(headers=["Food Item", "Calories"], type="array", value=[])
                    total_calories_label = gr.Label(label="Total Calories", value="0 kcal")

                add_food_btn.click(
                    fn=add_food_entry,
                    inputs=[food_item_input, food_calories_input],
                    outputs=[gr.Textbox(visible=False), food_log_list, total_calories_label]
                )

                clear_log_btn = gr.Button("Clear Log", variant="secondary")
                clear_log_btn.click(
                    fn=clear_food_log,
                    inputs=[],
                    outputs=[gr.Textbox(visible=False), food_log_list, total_calories_label]
                )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1784cae7aefd5849e5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
